<a href="https://colab.research.google.com/github/tejaswishetty17/Agentic-AI/blob/main/Fine_Tuning_a_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔧 Fine-Tuning DistilBERT for Sentiment Classification using PEFT and LoRA

In this tutorial, we will fine-tune a lightweight transformer model — `distilbert/distilbert-base-uncased` — for a binary sentiment classification task. Given a movie review, our model will predict whether the sentiment is **positive (1)** or **negative (0)**.

We'll use the **IMDB movie review dataset** from Hugging Face (`stanfordnlp/imdb`) and apply **Parameter-Efficient Fine-Tuning (PEFT)** techniques, specifically **LoRA (Low-Rank Adaptation)**, to reduce the number of trainable parameters and make the training more efficient, especially in resource-constrained environments like Google Colab.

By the end of this tutorial, you'll have a fine-tuned DistilBERT model capable of classifying movie reviews with high accuracy — using just a fraction of the original training cost.


In [1]:
!pip install -q transformers accelerate bitsandbytes peft datasets evaluate fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.1 MB/s eta 0:00:00


In [8]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [9]:
model_checkpoint = "distilbert-base-uncased"

id2label = {0: "Negative", 1:"Positive"}
label2id = {"Negative": 0, "Positive": 1}

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = 2, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Lets check how the base model performs for classification task

In [5]:
movie_reviews = [
    "This movie was an absolute masterpiece. The storytelling and acting were top-notch!",
    "I wasted two hours of my life. Poor plot, bad acting, and even worse dialogue.",
    "A visually stunning film with a powerful emotional core. I was completely immersed.",
    "The pacing was painfully slow and the plot went nowhere. Very disappointing.",
    "Fantastic performances and a beautiful soundtrack. Highly recommend!",
    "One of the worst movies I’ve seen this year. Not even worth watching for free.",
    "An emotional rollercoaster. It made me laugh, cry, and everything in between.",
    "Predictable and cliché. Felt like I had seen this exact movie ten times before.",
    "Brilliant direction and an unforgettable story. This one stays with you.",
    "Terrible editing and a lack of character development ruined what could have been decent."
]

for text in movie_reviews:
  inputs = tokenizer.encode(text, return_tensors="pt")
  logits = model(inputs).logits
  print(logits)
  predictions = torch.argmax(logits)
  print(text + " - " + id2label[predictions.tolist()])


tensor([[-0.1667, -0.1363]], grad_fn=<AddmmBackward0>)
This movie was an absolute masterpiece. The storytelling and acting were top-notch! - Positive
tensor([[-0.1818, -0.1419]], grad_fn=<AddmmBackward0>)
I wasted two hours of my life. Poor plot, bad acting, and even worse dialogue. - Positive
tensor([[-0.1725, -0.1233]], grad_fn=<AddmmBackward0>)
A visually stunning film with a powerful emotional core. I was completely immersed. - Positive
tensor([[-0.1750, -0.1248]], grad_fn=<AddmmBackward0>)
The pacing was painfully slow and the plot went nowhere. Very disappointing. - Positive
tensor([[-0.1927, -0.1135]], grad_fn=<AddmmBackward0>)
Fantastic performances and a beautiful soundtrack. Highly recommend! - Positive
tensor([[-0.1732, -0.1285]], grad_fn=<AddmmBackward0>)
One of the worst movies I’ve seen this year. Not even worth watching for free. - Positive
tensor([[-0.1585, -0.1183]], grad_fn=<AddmmBackward0>)
An emotional rollercoaster. It made me laugh, cry, and everything in between.

In [1]:
!pip install --upgrade datasets

In [ ]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("imdb")

dataset


In [3]:
dataset['train'][2000]

{'text': 'Darcy and her young daughter Pamela are heading out to the country where her mum\'s boyfriend Peter left his doctor\'s position in the city to become a writer and fix up a bed and breakfast inn. Although this inn has a terrible past and Pamela learns from one the girl\'s who lives in the town that a deformed witch once reside in that house. They called her the \'Tooth Fairy\' as she would kill kids after getting their last baby tooth. This work on the inn, has awoken the \'Tooth Fairy\'. Now she has her sights on Pamela and her last baby tooth, but if any gets in the way they face the same fate that awaits Pamela.<br /><br />This flick\'s old folk myth of the \'Tooth Fairy\' doesn\'t paint her in a very generous way, as you would believe when you were a child. Don\'t they just love turning happy childhood memories into nightmares! Another one which did fall into the same category was "Darkness Falls (2003)". I can\'t compare how similar they are in the premises, because I hav

In [4]:
train_small = dataset["train"].shuffle(seed=42).select(range(800))
test_small = dataset["test"].shuffle(seed=42).select(range(800))

dataset = DatasetDict({
    "train":train_small,
    "test":test_small
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 800
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 800
    })
})


In [5]:
dataset["train"][2]

{'text': 'George P. Cosmatos\' "Rambo: First Blood Part II" is pure wish-fulfillment. The United States clearly didn\'t win the war in Vietnam. They caused damage to this country beyond the imaginable and this movie continues the fairy story of the oh-so innocent soldiers. The only bad guys were the leaders of the nation, who made this war happen. The character of Rambo is perfect to notice this. He is extremely patriotic, bemoans that US-Americans didn\'t appreciate and celebrate the achievements of the single soldier, but has nothing but distrust for leading officers and politicians. Like every film that defends the war (e.g. "We Were Soldiers") also this one avoids the need to give a comprehensible reason for the engagement in South Asia. And for that matter also the reason for every single US-American soldier that was there. Instead, Rambo gets to take revenge for the wounds of a whole nation. It would have been better to work on how to deal with the memories, rather than suppressi

In [6]:
import numpy as np
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])

np.float64(0.49)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
#create tokenize function
def tokenize_function(examples):
  #extact text
  text = examples["text"]

  #tokenize and truncate text
  tokenizer.truncation_side = "left"
  tokenized_inputs = tokenizer(
      text,
      return_tensors = "np",
      truncation = True,
      max_length=512
  )

  return tokenized_inputs

#add pad token if none exists
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token':'[PAD]'})
  model.resize_token_embeddings(len(tokenizer))

#tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

In [ ]:
tokenized_dataset['train'][0]

In [13]:
# Create a dynamic data collator to pad sequences in each batch to the longest length
# Helps ensure efficient batching without padding the entire dataset to max length

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
#import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

# Define a metric function for the Trainer to evaluate model accuracy
# Applies argmax to predictions and compares with true labels

def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=1)

  return {"accuracy": accuracy.compute(predictions = predictions, references= labels)}

In [15]:
# Define LoRA config for parameter-efficient fine-tuning on a sequence classification task
# Applies LoRA to the 'q_lin' (query linear) layers with low-rank adaptation

peft_config = LoraConfig(
    task_type = "SEQ_CLS", # Indicates it's a sequence classification task.
    r = 4, #The rank of the LoRA decomposition (lower rank = fewer trainable params).
    lora_alpha = 32, #Scaling factor that controls the learning rate within LoRA layers.
    lora_dropout = 0.05, #Dropout applied to the LoRA layers to prevent overfitting.
    target_modules = ['q_lin'] #Specifies which layers in the model to apply LoRA to (e.g., the query linear layer in attention).
)

In [16]:
import warnings
warnings.filterwarnings("ignore")


# Apply LoRA configuration to the base model for efficient fine-tuning
# Print only trainable parameters

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [17]:
#hyper parameters
lr = 1e-3 #size of optimization step #.001
batch_size = 4 #number of examples processed per optimziation ste
num_epochs = 5 #number of times model runs through training data

#define training arguments
# define training arguments
training_args = TrainingArguments(
    output_dir= "distilbert-finetuned2",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # Fix 1: Disable Weights & Biases (wandb)
    report_to="none",
    # Fix 2: Explicitly provide label name
    #label_names=["labels"]
)

In [18]:
# creater trainer object
trainer = Trainer(
    model=model, # our peft model
    args=training_args, # hyperparameters
    train_dataset=tokenized_dataset["train"], # training data
    eval_dataset=tokenized_dataset["test"], # validation data
    tokenizer=tokenizer, # define tokenizer
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics, # evaluates model using compute_metrics() function from before
)

trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.324701,{'accuracy': 0.87125}
2,No log,0.489694,{'accuracy': 0.86125}
3,0.371400,0.700208,{'accuracy': 0.85875}
4,0.371400,0.673064,{'accuracy': 0.87}
5,0.110000,0.702759,{'accuracy': 0.865}


TrainOutput(global_step=1000, training_loss=0.24073773193359374, metrics={'train_runtime': 168.3579, 'train_samples_per_second': 23.759, 'train_steps_per_second': 5.94, 'total_flos': 442015308850944.0, 'train_loss': 0.24073773193359374, 'epoch': 5.0})

In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print(device)


movie_reviews = [
    "This movie was an absolute masterpiece. The storytelling and acting were top-notch!",
    "I wasted two hours of my life. Poor plot, bad acting, and even worse dialogue.",
    "A visually stunning film with a powerful emotional core. I was completely immersed.",
    "The pacing was painfully slow and the plot went nowhere. Very disappointing.",
    "Fantastic performances and a beautiful soundtrack. Highly recommend!",
    "One of the worst movies I’ve seen this year. Not even worth watching for free.",
    "An emotional rollercoaster. It made me laugh, cry, and everything in between.",
    "Predictable and cliché. Felt like I had seen this exact movie ten times before.",
    "Brilliant direction and an unforgettable story. This one stays with you.",
    "Terrible editing and a lack of character development ruined what could have been decent."
]

for text in movie_reviews:
  inputs = tokenizer.encode(text, return_tensors="pt").to(device)
  logits = model(inputs).logits
  predictions = torch.max(logits,1).indices
  print(text + " - " + id2label[predictions.tolist()[0]])

cuda
This movie was an absolute masterpiece. The storytelling and acting were top-notch! - Positive
I wasted two hours of my life. Poor plot, bad acting, and even worse dialogue. - Negative
A visually stunning film with a powerful emotional core. I was completely immersed. - Positive
The pacing was painfully slow and the plot went nowhere. Very disappointing. - Negative
Fantastic performances and a beautiful soundtrack. Highly recommend! - Positive
One of the worst movies I’ve seen this year. Not even worth watching for free. - Negative
An emotional rollercoaster. It made me laugh, cry, and everything in between. - Positive
Predictable and cliché. Felt like I had seen this exact movie ten times before. - Negative
Brilliant direction and an unforgettable story. This one stays with you. - Positive
Terrible editing and a lack of character development ruined what could have been decent. - Negative


In [20]:
model.save_pretrained("myfinetuned_model1")

 Zip the saved model folder

In [21]:
import shutil

shutil.make_archive("myfinetuned_model1", 'zip', "myfinetuned_model1")


'/content/myfinetuned_model1.zip'

Download the zipped file to your local machine

In [22]:
from google.colab import files

files.download("/content/myfinetuned_model1.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
model_checkpoint = "distilbert-base-uncased"

id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

base_model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
from peft import PeftModel
model = PeftModel.from_pretrained(base_model, "myfinetuned_model1")

In [35]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


print("Device used is ---", device)


movie_reviews = [
    "This movie was an absolute masterpiece. The storytelling and acting were top-notch!",
    "I wasted two hours of my life. Poor plot, bad acting, and even worse dialogue.",
    "A visually stunning film with a powerful emotional core. I was completely immersed.",
    "The pacing was painfully slow and the plot went nowhere. Very disappointing.",
    "Fantastic performances and a beautiful soundtrack. Highly recommend!",
    "One of the worst movies I’ve seen this year. Not even worth watching for free.",
    "An emotional rollercoaster. It made me laugh, cry, and everything in between.",
    "Predictable and cliché. Felt like I had seen this exact movie ten times before.",
    "Brilliant direction and an unforgettable story. This one stays with you.",
    "Terrible editing and a lack of character development ruined what could have been decent."
]

# for text in movie_reviews:
#   inputs = tokenizer.encode(text, return_tensors="pt").to(device)
#   logits = finetuned_model(inputs).logits
#   predictions = torch.argmax(logits)
#   print(text + " - " + id2label[predictions.tolist()])


for text in movie_reviews:
    inputs = tokenizer.encode(text, return_tensors="pt").to(device) # moving to mps for Mac (can alternatively do 'cpu')

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(text + " - " + id2label[predictions.tolist()[0]])

Device used is --- cuda
This movie was an absolute masterpiece. The storytelling and acting were top-notch! - Positive
I wasted two hours of my life. Poor plot, bad acting, and even worse dialogue. - Negative
A visually stunning film with a powerful emotional core. I was completely immersed. - Positive
The pacing was painfully slow and the plot went nowhere. Very disappointing. - Negative
Fantastic performances and a beautiful soundtrack. Highly recommend! - Positive
One of the worst movies I’ve seen this year. Not even worth watching for free. - Negative
An emotional rollercoaster. It made me laugh, cry, and everything in between. - Positive
Predictable and cliché. Felt like I had seen this exact movie ten times before. - Negative
Brilliant direction and an unforgettable story. This one stays with you. - Positive
Terrible editing and a lack of character development ruined what could have been decent. - Negative


In [36]:
#Training the model again after required steps.

#hyper parameters
lr = 1e-3 #size of optimization step #.001
batch_size = 4 #number of examples processed per optimziation ste
num_epochs = 5 #number of times model runs through training data

#define training arguments
# define training arguments
training_args = TrainingArguments(
    output_dir= "distilbert-finetuned2",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.02,
    #eval_strategy="epoch",
    #save_strategy="epoch",

    logging_strategy="steps",
    logging_steps=25,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=25,
    save_steps=25,
    load_best_model_at_end=True,
    # Fix 1: Disable Weights & Biases (wandb)
    report_to="none",
    # Fix 2: Explicitly provide label name
    #label_names=["labels"]

)


In [37]:
# creater trainer object
trainer = Trainer(
    model=model, # our peft model
    args=training_args, # hyperparameters
    train_dataset=tokenized_dataset["train"], # training data
    eval_dataset=tokenized_dataset["test"], # validation data
    tokenizer=tokenizer, # define tokenizer
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics, # evaluates model using compute_metrics() function from before
)

trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss,Accuracy
25,0.343400,0.429594,{'accuracy': 0.8675}
50,0.271200,0.415820,{'accuracy': 0.87125}
75,0.236400,0.606903,{'accuracy': 0.83625}
100,0.150900,0.505578,{'accuracy': 0.87}
125,0.057500,0.619315,{'accuracy': 0.855}
150,0.172300,0.601395,{'accuracy': 0.87125}
175,0.377100,0.996105,{'accuracy': 0.80875}
200,0.422500,0.751284,{'accuracy': 0.83875}
225,0.313500,0.587589,{'accuracy': 0.84375}
250,0.264500,0.577157,{'accuracy': 0.84875}


TrainOutput(global_step=1000, training_loss=0.24248940098285676, metrics={'train_runtime': 468.3785, 'train_samples_per_second': 8.54, 'train_steps_per_second': 2.135, 'total_flos': 442015308850944.0, 'train_loss': 0.24248940098285676, 'epoch': 5.0})

During training, we noticed that the validation loss increases while the training loss decreases.
This is a clear sign of overfitting, where the model memorizes training examples but fails to generalize to new, unseen data.

✅ Steps to Mitigate Overfitting (Hyperparameter Tuning)
To reduce overfitting, consider the following adjustments:

Reduce learning rate to 0.0001 for more stable training.

Increase weight decay to 0.05 to apply stronger regularization.

Use smaller batch sizes to introduce more gradient noise.

Limit the number of epochs to num_epochs = 3 to avoid over-training.

Increase training data size, ideally to at least 2000 examples, to improve generalization.

Applying these changes can help the model learn better representations and avoid overfitting.

In [38]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("imdb")

dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [39]:
train_small = dataset["train"].shuffle(seed=42).select(range(2000))
test_small = dataset["test"].shuffle(seed=42).select(range(2000))

dataset = DatasetDict({
    "train":train_small,
    "test":test_small
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [41]:
import numpy as np
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])

np.float64(0.5)

In [ ]:
#create tokenize function
def tokenize_function(examples):
  #extact text
  text = examples["text"]

  #tokenize and truncate text
  tokenizer.truncation_side = "left"
  tokenized_inputs = tokenizer(
      text,
      return_tensors = "np",
      truncation = True,
      max_length=512
  )

  return tokenized_inputs

#add pad token if none exists
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token':'[PAD]'})
  model.resize_token_embeddings(len(tokenizer))

#tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

In [43]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


accuracy = evaluate.load("accuracy")


def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=1)

  return {"accuracy": accuracy.compute(predictions = predictions, references= labels)}

In [44]:
peft_config = LoraConfig(
    task_type = "SEQ_CLS", # Indicates it's a sequence classification task.
    r = 4, #The rank of the LoRA decomposition (lower rank = fewer trainable params).
    lora_alpha = 16, #Scaling factor that controls the learning rate within LoRA layers.
    lora_dropout = 0.05, #Dropout applied to the LoRA layers to prevent overfitting.
    target_modules = ['q_lin'] #Specifies which layers in the model to apply LoRA to (e.g., the query linear layer in attention).
)

In [45]:
#hyper parameters
lr = 1e-4 #size of optimization step #.001
batch_size = 8
num_epochs = 2


training_args = TrainingArguments(
    output_dir= "distilbert-finetuned2",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.02,
    #eval_strategy="epoch",
    #save_strategy="epoch",

    logging_strategy="steps",
    logging_steps=25,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=25,
    save_steps=25,
    load_best_model_at_end=True,
    # Fix 1: Disable Weights & Biases (wandb)
    report_to="none",


)


In [46]:
# creater trainer object
trainer = Trainer(
    model=model, # our peft model
    args=training_args, # hyperparameters
    train_dataset=tokenized_dataset["train"], # training data
    eval_dataset=tokenized_dataset["test"], # validation data
    tokenizer=tokenizer, # define tokenizer
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics, # evaluates model using compute_metrics() function from before
)

trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss,Accuracy
25,0.308000,0.369259,{'accuracy': 0.876}
50,0.286800,0.360298,{'accuracy': 0.8745}
75,0.268900,0.355395,{'accuracy': 0.874}
100,0.196100,0.356086,{'accuracy': 0.8755}
125,0.261400,0.353932,{'accuracy': 0.877}
150,0.292300,0.350319,{'accuracy': 0.877}
175,0.294700,0.345196,{'accuracy': 0.8765}
200,0.372800,0.337940,{'accuracy': 0.877}
225,0.363900,0.331320,{'accuracy': 0.877}
250,0.218000,0.331005,{'accuracy': 0.878}


TrainOutput(global_step=500, training_loss=0.2777618408203125, metrics={'train_runtime': 619.2031, 'train_samples_per_second': 6.46, 'train_steps_per_second': 0.807, 'total_flos': 497497171889664.0, 'train_loss': 0.2777618408203125, 'epoch': 2.0})

In [47]:
model.save_pretrained("final_finetuned_tuned")

In [48]:
model_checkpoint = "distilbert-base-uncased"

id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

base_model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
from peft import PeftModel
model = PeftModel.from_pretrained(base_model, "final_finetuned_tuned")

In [50]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


print("Device used is ---", device)


movie_reviews = [
    "This movie was an absolute masterpiece. The storytelling and acting were top-notch!",
    "I wasted two hours of my life. Poor plot, bad acting, and even worse dialogue.",
    "A visually stunning film with a powerful emotional core. I was completely immersed.",
    "The pacing was painfully slow and the plot went nowhere. Very disappointing.",
    "Fantastic performances and a beautiful soundtrack. Highly recommend!",
    "One of the worst movies I’ve seen this year. Not even worth watching for free.",
    "An emotional rollercoaster. It made me laugh, cry, and everything in between.",
    "Predictable and cliché. Felt like I had seen this exact movie ten times before.",
    "Brilliant direction and an unforgettable story. This one stays with you.",
    "Terrible editing and a lack of character development ruined what could have been decent."
]


for text in movie_reviews:
    inputs = tokenizer.encode(text, return_tensors="pt").to(device) # moving to mps for Mac (can alternatively do 'cpu')

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(text + " - " + id2label[predictions.tolist()[0]])

Device used is --- cuda
This movie was an absolute masterpiece. The storytelling and acting were top-notch! - Positive
I wasted two hours of my life. Poor plot, bad acting, and even worse dialogue. - Negative
A visually stunning film with a powerful emotional core. I was completely immersed. - Positive
The pacing was painfully slow and the plot went nowhere. Very disappointing. - Negative
Fantastic performances and a beautiful soundtrack. Highly recommend! - Positive
One of the worst movies I’ve seen this year. Not even worth watching for free. - Negative
An emotional rollercoaster. It made me laugh, cry, and everything in between. - Positive
Predictable and cliché. Felt like I had seen this exact movie ten times before. - Negative
Brilliant direction and an unforgettable story. This one stays with you. - Positive
Terrible editing and a lack of character development ruined what could have been decent. - Negative
